In [1]:
import numpy as np
from plotly import express as px
from plotly import subplots as sp
from plotly import graph_objects as go
from matplotlib import pyplot as plt
import pandas as pd
import sys
import os
import wandb

import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torch.utils.data import random_split
from torch.utils.data import DataLoader

import torchvision.transforms as transforms

AssignDIR = os.path.dirname(os.path.dirname(os.path.abspath('CNN.ipynb')))
CurrDIR = os.path.dirname(os.path.abspath('CNN.ipynb'))
UserDIR = os.path.dirname(AssignDIR)

sys.path.append(UserDIR)

RawDataDIR = os.path.join(UserDIR, "./data/external/")
PreProcessDIR = os.path.join(UserDIR, "./data/interim/4/")

from utils_4 import *
from models.cnn import cnn
from models.cnn import multilabel_cnn

In [2]:
dataloader_multi = MultiMNISTDataset(32)
dataloader_multi.load_mnist_data('multi_label')

In [3]:
train_labels = dataloader_multi.OneHotEncoder(dataloader_multi._train_labels)
valid_labels = dataloader_multi.OneHotEncoder(dataloader_multi._valid_labels)
test_labels = dataloader_multi.OneHotEncoder(dataloader_multi._test_labels)

In [4]:
config_model = {'task': [], 'in_channels': [], 'ConvLayers': [], 'FCLayers': [], 'pool': [], 
                'kernel_size': [], 'strides': [], 'activation': [], 'dropout': []}

config_trainer = {'model': [], 'loss': [], 'epochs': [], 'batch_size': [], 'learning_rate': [], 
                  'optimizer': [], 'wandb': False, 'labels': [], 'labelsRnum': True, 'FM_Vis': False}

In [5]:
config_model['task'] = 'classification'
config_model['in_channels'] = 1
config_model['ConvLayers'] = [32, 64]
config_model['FCLayers'] = [64*7*7, 512, 33]
config_model['kernel_size'] = [3, 3, 3]
config_model['strides'] = [1, 1, 1]
config_model['pool'] = [2, 2, 2]
config_model['activation'] = 'relu'
config_model['dropout'] = 0.3

model = multilabel_cnn.MultiCNN(config_model)

config_trainer['model'] = model
config_trainer['loss'] = 'BCE'
config_trainer['epochs'] = 10
config_trainer['batch_size'] = 256
config_trainer['learning_rate'] = 1e-5
config_trainer['optimizer'] = optim.Adam(model.parameters(), lr=config_trainer['learning_rate'])
config_trainer['wandb'] = False
config_trainer['labels'] = dataloader_multi.getLabels('multi_label')
config_trainer['labelsRnum'] = False
config_trainer['FM_Vis'] = False

trainer = multilabel_cnn.Model_trainer(config_trainer)
trainer.trainer(dataloader_multi._train_images, train_labels, 
                dataloader_multi._valid_images, valid_labels, 5)

Epoch:  0
Acc =  0.40057291666666667
Loss =  0.72965956
Acc =  0.4111328125
Loss =  0.72959906
Epoch:  1
Acc =  0.4292187500000001
Loss =  0.7235325
Acc =  0.4373914930555556
Loss =  0.7232525
Epoch:  2
Acc =  0.433984375
Loss =  0.71364695
Acc =  0.439236111111111
Loss =  0.7130387
Epoch:  3
Acc =  0.4327083333333333
Loss =  0.70947266
Acc =  0.43901909722222215
Loss =  0.7087706
Epoch:  4
Acc =  0.4350260416666667
Loss =  0.70764697
Acc =  0.439236111111111
Loss =  0.70693016
